In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

base: str = "./datasets/"

# Load Transactions dataset

In [6]:
transactions = pd.read_csv(
    base + "transactions.csv",
    names=['timestamp', 'blockId', 'txId', 'isCoinbase', 'fee'],
    dtype={'blockId': np.int32, 'txId': np.int32, 'isCoinbase': np.int8},
    parse_dates=True
)

#print(transactions.info())
#print(pd.concat([transactions.head(), transactions.tail()]))
#print(transactions.head())
#print(transactions.tail())

'''
max vals
timestamp: 1356997957 -> int64
blockId: 214562 -> int32
txId: 10572826 -> int32
isCoinbase -> np.bool_
fee: 17179869184 -> int64
'''

#print(transactions.nlargest(5, 'fee'))


'\nmax vals\ntimestamp: 1356997957 -> int64\nblockId: 214562 -> int32\ntxId: 10572826 -> int32\nisCoinbase -> np.bool_\nfee: 17179869184 -> int64\n'

# Load Inputs dataset

In [7]:
inputs = pd.read_csv(
    base + "inputs.csv",
    names=['txId', 'prevTxId', 'prevTxPos'],
    dtype={'txId': np.int32, 'prevTxId': np.int32, 'prevTxPos': np.int16},
)

# Load Outputs dataset 

In [8]:
outputs = pd.read_csv(
    base + "outputs.csv",
    names=['txId', 'txPos', 'addressId', 'amount', 'scriptType'],
    dtype={'txId': np.int32, 'txPos': np.int16, 'addressId': np.int32, 'amount': np.int64, 'scriptType': 'category'}
)

#print(outputs.info())
#print(outputs)

# Load Mappings dataset

In [ ]:
mappings = pd.read_csv(
    base + "mappings.csv",
    names=['addressId', 'hash'],
    dtype={'addressId': str, 'hash': np.int32}
)

# Script table

In [ ]:
data = {'scriptCode': [0,1,2,3,4,5,6,7], 
        'scriptType':["Unknown", "P2PK", "P2KH", "P2SH", "RETURN", "EMPTY", "P2WPKH", "P2WSH"],
        'scriptSize': [0, 153, 180, 291, 0, 0, 0, 0]
       }

scripts = pd.DataFrame(
    data,
    #dtypes={'scriptCode': np.int8, 'scriptType': str, 'scriptSize': np.int8}
                  
)
print(scripts.info())
print(scripts)


# Size of transactions

In [ ]:
#Df to have the num of transactions for every timestamp
transCount = transactions \
    .groupby('timestamp')['txId'].count() \
    .reset_index() \
    .rename(columns={'txId':'txCount'})
print(transCount)

transCount.plot.line(title='Numero di transazioni nel tempo', x='timestamp', y='txCount')

# Utils

In [9]:
def getTransactionInputs(inputsDf: pd.DataFrame, txId: int) -> pd.DataFrame:
    ''' Get all inputs of a transaction 
    
    Returns:
    DF<txId, prevTxId, prevTxPos>'''

    cond = inputsDf['txId'] == txId
    return inputsDf[cond]

def getTransactionOutputs(outputsDf: pd.DataFrame, txId: int) -> pd.DataFrame:
    ''' Get all outputs of a transaction 
    
    Returns:
    DF<txId, txPos, addressId, amount, scriptType>'''
    
    cond = outputsDf['txId'] == txId
    return outputsDf[cond]

def getTransactionSize(inputsDf: pd.DataFrame, outputsDf: pd.DataFrame, txId: int) -> float:
    ''' Ottieni la dimensione di una transazione '''
    
    INPUT_SIZE = 40
    OUTPUT_SIZE = 9

    num_inputs = len(getTransactionInputs(inputsDf, txId))
    num_outputs = len(getTransactionOutputs(outputsDf, txId))
    script_size = 0

    # TODO: num_outputs > 1 ==> script diversi; quale contare?
    return INPUT_SIZE * num_inputs + OUTPUT_SIZE * num_outputs + script_size

def getTransactionsAtTimestamp(transactionsDf: pd.DataFrame, timestamp: int) -> pd.DataFrame:
    """ Get all transaction at a given timestamp 
    
    Returns:
    DF<timestamp, blockId, txId, isCoinbase, fee>"""

    cond = transactionsDf['timestamp'] == timestamp
    return transactionsDf[cond]


def getCongestionAtTs(timestamp: int, transactionsDf: pd.DataFrame, inputsDf: pd.DataFrame, outputsDf: pd.DataFrame) -> int:
    """ Get the congestion at a given timestamp """
    
    transactions_at_ts = getTransactionsAtTimestamp(transactionsDf, timestamp).drop(columns=["blockId", "isCoinbase", "fee"])
    # transactions_at_ts: <timestamp, txId>

    tx_sizes: list[float] = [getTransactionSize(inputsDf, outputsDf, txId) for txId in transactions_at_ts['txId']]
    return sum(tx_sizes)


def getAverageFeeAtTimestamp(timestamp: int, transactionsDf: pd.DataFrame) -> float:
    tx_ts: pd.DataFrame = getTransactionsAtTimestamp(transactionsDf, timestamp)

    return tx_ts['fee'].mean()


def getTsCongestionDf(transactionsDf: pd.DataFrame, inputsDf: pd.DataFrame, outputsDf: pd.DataFrame) -> pd.DataFrame:
    ''' STEPS:
    Filter transactionsDf (remove coinbases)
    Get timestamps
    Get congestion at timestamp
    Get fee at timestamp
    '''
    
    not_coinbase = transactionsDf['isCoinbase'] == 0
    transactionsDf = transactionsDf[not_coinbase]

    n = 572828
    
    timestamps = transactionsDf['timestamp'][n:n+500]
    congs = timestamps.apply(getCongestionAtTs, args=(transactionsDf, inputsDf, outputsDf))
    fees = timestamps.apply(getAverageFeeAtTimestamp, args=(transactionsDf))
    
    return pd.DataFrame({'Timestamp': timestamps, 'Congestion': congs, 'Fee': fees})

In [18]:
# test getTransactionInputs, getTransactionOutputs
TX_ID = 10888

txInputs = getTransactionInputs(inputs, TX_ID)
print(f'#tx_inputs: {len(txInputs)}')
print(txInputs)

txOutputs = getTransactionOutputs(outputs, TX_ID)
print(f'#tx_outputs: {len(txOutputs)}')
print(txOutputs)

#tx_inputs: 1
      txId  prevTxId  prevTxPos
640  10888     10760          0
#tx_outputs: 2
        txId  txPos  addressId      amount  scriptType
10915  10888      0      10849  1000000000           2
10916  10888      1      10722  4000000000           1


In [ ]:
#get all transactions with more then 1 output
'''
for id in transactions['txId']:
    n_outputs = len(getTransactionOutputs(outputs, id))

    if n_outputs != 1:
        print(f'txId:{id:}, num_outputs: {n_outputs}')

print("end")'''

In [11]:
# test getTransactionsAtTimestamp
timestamp = 1356997591
transactions_ncb = transactions[transactions['isCoinbase'] == 0]

tx_ts = getTransactionsAtTimestamp(transactions_ncb, timestamp)

print(tx_ts)

NameError: name 'getTransactionsAtTimestamp' is not defined

In [20]:
# test getCongestionAtTs
congestion = getCongestionAtTs(timestamp, transactions_ncb, inputs, outputs)
print(f'congestion @{timestamp}: {congestion}')


congestion @1356997591: 11358


In [23]:
# test getAverageFeeAtTimestamp
fee = getAverageFeeAtTimestamp(timestamp, transactions_ncb)
print(f'avg fee @{timestamp}: {fee}')

avg fee @1356997591: 39178.82269503546


In [10]:
# fee test
from datetime import datetime

ts_start = 1231006505
ts_end = 1356997957

random_ts = transactions.sample(n=5)['timestamp'].to_list()
print(random_ts)
#ts = random.randint(ts_start, ts_end)

for ts in random_ts:
    fee = getAverageFeeAtTimestamp(ts, transactions)
    print(f'avg fee @{ts}: {fee}')


[1322809509, 1323167912, 1347854040, 1314461706, 1348265435]
avg fee @1322809509: 65728.78651685393
avg fee @1323167912: 60224.313725490196
avg fee @1347854040: 58870.967741935485
avg fee @1314461706: 25036.969325153375
avg fee @1348265435: 54290.45735294118


In [11]:
transactions_ncb = transactions[transactions['isCoinbase'] == 0]

tt = transactions_ncb.groupby('timestamp')['fee'].mean()

print(tt)

#tt.plot('line')

timestamp
1231731025        0.000000
1231740133        0.000000
1231740736        0.000000
1231742062        0.000000
1231744600        0.000000
                  ...     
1356995372     2500.000000
1356996072    86570.743405
1356996754    83062.015504
1356997591    39178.822695
1356997957    83861.003861
Name: fee, Length: 130620, dtype: float64


In [13]:

merged_tx_inputs = pd.merge(transactions_ncb, inputs, on='txId', how='inner')

# Merge transactions with outputs
merged_tx_outputs = pd.merge(transactions_ncb, outputs, on='txId', how='inner')

# Calculate size for inputs and outputs separately
size_inputs = merged_tx_inputs.groupby('txId').size().reset_index(name='num_inputs')
size_outputs = merged_tx_outputs.groupby('txId').size().reset_index(name='num_outputs')

print(size_inputs)
print(size_outputs)

              txId  num_inputs
0              171           1
1              183           1
2              185           1
3              187           1
4              192           1
...            ...         ...
10358261  10572822           1
10358262  10572823           1
10358263  10572824           1
10358264  10572825           1
10358265  10572826           2

[10358266 rows x 2 columns]
              txId  num_outputs
0                0            1
1                1            1
2                2            1
3                3            1
4                4            1
...            ...          ...
10572822  10572822            2
10572823  10572823            2
10572824  10572824            2
10572825  10572825            2
10572826  10572826            3

[10572827 rows x 2 columns]


In [19]:
# Merge the sizes with transactions
merged_transactions = pd.merge(transactions_ncb, size_inputs, on='txId', how='left')
merged_transactions = pd.merge(merged_transactions, size_outputs, on='txId', how='left')
merged_transactions = merged_transactions.drop(columns=['blockId', 'isCoinbase'])

INPUT_SIZE = 40
OUTPUT_SIZE = 9

# Calculate size of each transaction
merged_transactions['transaction_size'] = merged_transactions['num_inputs'] * INPUT_SIZE + merged_transactions['num_outputs'] * OUTPUT_SIZE
merged_transactions = merged_transactions.drop(columns=['num_inputs', 'num_outputs'])

print(merged_transactions)

           timestamp      txId     fee  transaction_size
0         1231731025       171       0                58
1         1231740133       183       0                58
2         1231740736       185       0                58
3         1231742062       187       0                58
4         1231744600       192       0                49
...              ...       ...     ...               ...
10358261  1356997957  10572822  100000                58
10358262  1356997957  10572823  100000                58
10358263  1356997957  10572824  100000                58
10358264  1356997957  10572825  100000                58
10358265  1356997957  10572826  100000               107

[10358266 rows x 4 columns]


In [28]:
tt = merged_transactions.groupby('timestamp').agg({'fee': 'mean', 'transaction_size': 'sum'}).reset_index()
tt = tt.rename(columns={'fee': 'avgFee', 'transaction_size': 'congestion'})
print(tt)

tt.plot.line(x='timestamp', y='avgFee', title='Andamento avgFee nel tempo')
tt.plot.line(x='timestamp', y='congestion', title='Andamento congestione nel tempo')


         timestamp        avgFee  congestion
0       1231731025      0.000000          58
1       1231740133      0.000000          58
2       1231740736      0.000000          58
3       1231742062      0.000000          58
4       1231744600      0.000000          49
...            ...           ...         ...
130615  1356995372   2500.000000         766
130616  1356996072  86570.743405       45913
130617  1356996754  83062.015504       30225
130618  1356997591  39178.822695       11358
130619  1356997957  83861.003861       25663

[130620 rows x 3 columns]


In [ ]:
tt.plot.hist('avgFee', bins=5000)